# Sending Bitcoin!

#### Send 0.04 TBTC to this address `muvpVznkBtk8rRSxLRVQRdUhsMjS7aKRne`

#### Go here to send your transaction: https://testnet.blockexplorer.com/tx/send or https://live.blockcypher.com/btc-testnet/pushtx/

In [20]:
from ecc import PrivateKey
from helper import decode_base58, p2pkh_script, SIGHASH_ALL
from script import Script
from tx import TxIn, TxOut, Tx

whole = 100000000
prev_tx = bytes.fromhex('e4a46885ef4066bf070bf6b5daa488cd0fee1fbc806ac45b0e145cdcd719c7cd')
prev_index = 1 # FILL THIS IN
target_address = 'muvpVznkBtk8rRSxLRVQRdUhsMjS7aKRne'
in_amount = 78817625 / whole
target_amount = 70000000 / whole
fee = 0.008
change_address = 'mgTY8Fq2XstGZuf5AvqWTiUu911PGHx9N8'
change_amount = in_amount - fee - target_amount # CALCULATE THIS
secret = 14322962188143229621881
priv = PrivateKey(secret=secret)

print(target_amount, fee, change_amount)

# initialize inputs
tx_ins = []
# create a new tx input with prev_tx, prev_index, blank script_sig and max sequence
tx_ins.append(TxIn(
            prev_tx=prev_tx,
            prev_index=prev_index,
            script_sig=b'',
            sequence=0xffffffff,
        ))

# initialize outputs
tx_outs = []
# decode the hash160 from the target address
h160 = decode_base58(target_address)
# convert hash160 to p2pkh script
script_pubkey = p2pkh_script(h160)
# convert target amount to satoshis (multiply by 100 million)
target_satoshis = int(target_amount*100000000)
# create a new tx output for target with amount and script_pubkey
tx_outs.append(TxOut(
    amount=target_satoshis,
    script_pubkey=script_pubkey,
))
# decode the hash160 from the change address
h160 = decode_base58(change_address)
# convert hash160 to p2pkh script
script_pubkey = p2pkh_script(h160)
# convert change amount to satoshis (multiply by 100 million)
change_satoshis = int(change_amount*100000000)
# create a new tx output for target with amount and script_pubkey
tx_outs.append(TxOut(
    amount=change_satoshis,
    script_pubkey=script_pubkey,
))

# create the transaction
tx_obj = Tx(version=1, tx_ins=tx_ins, tx_outs=tx_outs, locktime=0, testnet=True)

# now sign the 0th input with the private key using SIGHASH_ALL using sign_input
tx_obj.sign_input(0, priv, SIGHASH_ALL)

# SANITY CHECK: change address corresponds to private key
if priv.point.address(testnet=True) != change_address:
    raise RuntimeError('Private Key does not correspond to Change Address, check priv_key and change_address')

# SANITY CHECK: output's script_pubkey is the same one as your address
if tx_ins[0].script_pubkey(testnet=True).elements[2] != decode_base58(change_address):
    raise RuntimeError('Output is not something you can spend with this private key. Check that the prev_tx and prev_index are correct')

# SANITY CHECK: fee is reasonable
if tx_obj.fee() > 0.05*100000000 or tx_obj.fee() <= 0:
    raise RuntimeError('Check that the change amount is reasonable. Fee is {}'.format(tx_obj.fee()))

# serialize and hex()
print(tx_obj.serialize().hex())


0.7 0.008 0.08017625000000006
0100000001cdc719d7dc5c140e5bc46a80bc1fee0fcd88a4dab5f60b07bf6640ef8568a4e4010000006b483045022100dbd3d97d8f12623ce6469126977185d26e44faa7196c4609f40a09a4eb827cc50220710737dc7d36bc9823312391e8521dcfc9506a739d423beb3129d567422b69c7012102a8232e5a011ba556dd57483296790f7131fc6d842626d7f74a4e1b008d0235faffffffff02801d2c04000000001976a9149e153ce5de3689901301309b9e8d804ab4804d4b88acd9567a00000000001976a9140a52b3e3f855cc255c94f7be40f1bffeb867497488ac00000000


### Bonus:

Get some testnet coins and spend both outputs (one from your change address and one from the testnet faucet) to `muvpVznkBtk8rRSxLRVQRdUhsMjS7aKRne`

#### You can get some free testnet coins at: https://testnet.coinfaucet.eu/en/

In [2]:
# add a 3rd input & 3rd output
# use cltv on all outputs
# from tx use fee function to verify fee amount 
#reconcile estimated fee with actual
# Bonus

from ecc import PrivateKey
from helper import decode_base58, p2pkh_script, SIGHASH_ALL
from script import Script
from tx import TxIn, TxOut, Tx

prev_tx_1 = bytes.fromhex('6a66363cbccfa5bb43bea30af87fb3cc46092a672388e1631e555adb00408b13')
prev_index_1 = 0
prev_tx_2 = bytes.fromhex('22f8a9803376329343f4881727d4dec2c24a0f5dde5ee001911fb54ee0c246fd')
prev_index_2 = 1
#added  3rd tx
prev_tx_3 = bytes.fromhex('4c660e4fa7e5470cb21f462cb5bfadd10031e68d4aaf4640dcfb958e4c124a46')
prev_index_3 = 1
#added fourth tx
prev_tx_4 = bytes.fromhex('1c41531787fe6992afb7fa2e042ac76b80daf56fe016544dea09500695fc1851')
prev_index_4 = 1 
target_address = 'muvpVznkBtk8rRSxLRVQRdUhsMjS7aKRne'
target_amount = .01
secret = 14322962188143229621881
change_address = 'mgTY8Fq2XstGZuf5AvqWTiUu911PGHx9N8'

#added third target
target_address_2 = 'myFro1vYkUNuKSSYsxsxKoJKua4yKs2wP8'
target_amount_2 = .01

# changed to output $ amounts for fees and change
whole = 100000000
est_fee = 4800 / whole
in_amount = .10260991 
change_amount = 543000 / 100000000
priv = PrivateKey(secret=secret)
btc_price = 6500

print('fee: $' + str(btc_price*est_fee) + '/' + str(est_fee*whole) + '(satoshis)')
print('sending: $' + str(btc_price *target_amount))
print('change: $' + str(btc_price * change_amount) + '/' + str(change_amount * whole) + '(satoshis)')

#added to calculate 4 sat per byte fee
sat_per_byte = 4
def utf8len(s):
    right_fee = (len(s.encode('utf-8')) * sat_per_byte) 
    return right_fee



# initialize inputs
tx_ins = []

# create the 1st tx input with prev_tx_1, prev_index_1, blank script_sig and max sequence
tx_ins.append(TxIn(
    prev_tx=prev_tx_1,
    prev_index=prev_index_1,
    script_sig=b'',
    sequence=0xffffffff,
)) 


# deleted an input / was getting "Error validating transaction: Rejected script for input..." error when submitting hex tx


# added: create the third tx input with prev_tx_3, prev_index_3, blank script_sig and max sequence
tx_ins.append(TxIn(
    prev_tx=prev_tx_3,
    prev_index=prev_index_3,
    script_sig=b'',
    sequence=0xffffffff,
)) 

# added: create the 4th tx input with prev_tx_4, prev_index_4, blank script_sig and max sequence
tx_ins.append(TxIn(
    prev_tx=prev_tx_4,
    prev_index=prev_index_4,
    script_sig=b'',
    sequence=0xffffffff,
))


# initialize outputs
tx_outs = []
# decode the hash160 from the target address
h160 = decode_base58(target_address)
# convert hash160 to p2pkh script
script_pubkey = p2pkh_script(h160)
# convert target amount to satoshis (multiply by 100 million)
target_satoshis = int(target_amount*100000000)
# create a single tx output for target with amount and script_pubkey
tx_outs.append(TxOut(
    amount=target_satoshis,
    script_pubkey=script_pubkey,
))

# added another  tx_out / decode the hash160 from the target address
h160 = decode_base58(target_address_2)
# convert hash160 to p2pkh script
script_pubkey = p2pkh_script(h160)
# convert target amount to satoshis (multiply by 100 million)
target_satoshis_2 = int(target_amount_2*100000000)
# create a single tx output for target with amount and script_pubkey
tx_outs.append(TxOut(
    amount=target_satoshis_2,
    script_pubkey=script_pubkey,
))

# decode the hash160 from the change address
h160 = decode_base58(change_address)
# convert hash160 to p2pkh script
script_pubkey = p2pkh_script(h160)
# convert change amount to satoshis (multiply by 100 million)
change_satoshis = int(change_amount*100000000)
# create a new tx output for target with amount and script_pubkey
tx_outs.append(TxOut(
    amount=change_satoshis,
    script_pubkey=script_pubkey,
))



# create the transaction
tx_obj = Tx(1, tx_ins, tx_outs, 0, testnet=True)

# sign both (3) inputs with the private key using SIGHASH_ALL using sign_input
tx_obj.sign_input(0, priv, SIGHASH_ALL)
tx_obj.sign_input(1, priv, SIGHASH_ALL)
tx_obj.sign_input(2, priv, SIGHASH_ALL)

# SANITY CHECK: output's script_pubkey is the same one as your address
if tx_ins[0].script_pubkey(testnet=True).elements[2] != decode_base58(priv.point.address(testnet=True)):
    raise RuntimeError('Output is not something you can spend with this private key. Check that the prev_tx and prev_index are correct')

# SANITY CHECK: fee is reasonable
if tx_obj.fee() > 0.05*100000000 or tx_obj.fee() <= 0:raise RuntimeError('Check that the change amount is reasonable. Fee is {}'.format(tx_obj.fee()))

# added: print actual fee
print('Actual fee is: ' + str(tx_obj.fee()) + ' (satoshis)')
# serialize and hex()
tx = tx_obj.serialize().hex()
#calcualate reccomended fee
print('Recc. Fee(satoshis): ' + str(utf8len(tx)) + '\n')
# print hex tx
print(tx)






fee: $0.312/4800.0(satoshis)
sending: $65.0
change: $35.295/543000.0(satoshis)
Actual fee is: 7100 (satoshis)
Recc. Fee(satoshis): 4440

0100000003138b4000db5a551e63e18823672a0946ccb37ff80aa3be43bba5cfbc3c36666a000000006b483045022100951909608f6f8b243ac52252b1e980902d468b22c427890d6e4b92689c19db9902202093cfb2fd5d4679f1617f06eceacb4689baecc4d5dbef8fdb6758440961c937012102a8232e5a011ba556dd57483296790f7131fc6d842626d7f74a4e1b008d0235faffffffff464a124c8e95fbdc4046af4a8de63100d1adbfb52c461fb20c47e5a74f0e664c010000006a47304402206318ec5b254b27069ddf43a970aac30c7818968f2c088a429e99fbbe9b1ecc97022020d8e16746f4c0787c0b8d4ef67b9e26cd703ce649bcee76d44c098b55562ddd012102a8232e5a011ba556dd57483296790f7131fc6d842626d7f74a4e1b008d0235faffffffff5118fc95065009ea4d5416e06ff5da806bc72a042efab7af9269fe871753411c010000006b483045022100bf3303faa38b0f52e58cf227560977224cf1e7c6c54bdebd780e0b17176f7d140220072689d43b4acb92c9497c5c657f98c9dc3187244b3c7d9db02aceb0ee8d32c3012102a8232e5a011ba556dd57483296790f7131fc6d8